# AMOS 2024 - BARRA2 historical Heatwave Case Study

AUTHOR: Chun-Hsu Su (chunhsu.su@bom.gov.au)

In this notebook we demonstrate the use of BARRA2 data in the NCI Data Collection to explore the NSW heatwave of January 2017.

More information on BARRA2 data: https://opus.nci.org.au/pages/viewpage.action?pageId=264241166

Before using this notebook, users must join ob53 project via, https://my.nci.org.au/mancini/project/ob53/join

Reanalyses are useful for users to go back in time to look at how a past weather event had occurred and the driving atmospheric processes.

***

This case study looks at the early 2017 New South Wales heatwaves. 

A special climate statement reporting on the exceptional heat in southeast Australia can be found http://www.bom.gov.au/climate/current/statements/scs61.pdf

In January and February, there were three distinct heatwaves in southeast Australia, with the highest temperatures recorded over 9–12 February 2017. 

There were 3 heatwaves across January and early February 2017 saw unusually high daily maximum and minimum temperatures for at least three consecutive days over large parts of the country. The first heatwave began around 10 January and continued to 14 January. 

The pattern of pressure systems - high pressure system over Tasman Sea and an upper level ridge over central and eastern Australia - drew hot air from central part of the continent.


In [ ]:
import os
import nci_ipynb
os.chdir(nci_ipynb.dir())
print(os.getcwd())

In [ ]:
# Import standard python modules
import os, sys
from datetime import datetime
import dask.distributed
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)

import esmloader

In [ ]:
# Specify the which data collection you want to load, i.e. BARRA2 or BARPA.
barra2 = esmloader.EsmCat("BARRA2")

In [ ]:
# Start a dask client
client = dask.distributed.Client()
client

### 1. Explore the BARRA-R2 data sets

What experiments are available in BARRA2 at this time?

At this time, we have BARRA-R2 12 km reanalysis data. 

Later this year, the BARRA-RE2 ensemble and BARRA-C2 4.4 km reanalysis data will be available. BARRA-C2 will appear as "AUS-04  hres  BARRA-C2" because it is defined over a different spatial domain AUS-04 and nested in ERA5 HRES analysis.

In [ ]:
barra2.keys

In [ ]:
#loaddata.list_experiments("BARRA2")
print(barra2.get_values('domain_id'))
print(barra2.get_values('driving_variant_label'))
print(barra2.get_values('source_id'))

What hourly variables are available?

In [ ]:
_ = barra2.list_barra2_variables("BARRA-R2", "1hr")

For heatwave case study, we will look at the screen-level temperature, near-surface wind vector and low level cloud cover.

First check they are what they are, and explore different variable options

In [ ]:
_ = barra2.whatis("1hr", "tas")

In [ ]:
_ = barra2.whatis("1hr", "tasmax")

In [ ]:
_ = barra2.whatis("1hr", "uas")

In [ ]:
_ = barra2.whatis("1hr", "uasmax")

In [ ]:
_ = barra2.whatis("1hr", "vas")

In [ ]:
_ = barra2.whatis("1hr", "cll")

In [ ]:
_ = barra2.whatis("fx", "orog")

In [ ]:
_ = barra2.whatis("fx", "sftlf")

For this case study, we will use
- hourly maximum temperature (1hr/tasmax)
- hourly inst u and v (1hr/uas, 1hr/vas)
- hourly mean low-level cloud cover (1hr/cll)

We will use the loaddata.load_barra2_data function.

In [ ]:
help(barra2.load_barra2_data)

### 2. First look at the temperature variability in the region around Sydney during December 2016 and March 2017

In [ ]:
# Location of Sydney
loc = (-33.8688, 151.2093)

# Examine a small 2x2 degrees bounding box around Sydney
latmin_c = loc[0] - 1
latmax_c = loc[0] + 1
lonmin_c = loc[1] - 1
lonmax_c = loc[1] + 1

# Look at Dec 2016 to Mar 2017 time period. The time format is yyyymmdd, yyyy is year, mm is month, dd is day
tstart = 20161201
tend = 20170331

We compute spatial mean of tasmax over during the few months and pick a smaller time period to examine in the next step.

In [ ]:
# Load tasmax data
ds_tasmax = barra2.load_barra2_data("BARRA-R2", "1hr", "tasmax", 
                                    tstart=tstart, tend=tend,
                                    latrange=(latmin_c, latmax_c),
                                    lonrange=(lonmin_c, lonmax_c))

# Convert to degC and update the units attributes
ds_tasmax['tasmax'] = ds_tasmax['tasmax'] - 273.15
ds_tasmax['tasmax'] = ds_tasmax['tasmax'].assign_attrs({"units": "degC"})
(NT, NY, NX) = ds_tasmax['tasmax'].shape

# Load land sea mask
ds_lsm = barra2.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                 latrange=(latmin_c, latmax_c),
                                 lonrange=(lonmin_c, lonmax_c))

# Generate a 3d land sea mask
mask_condition = np.tile(ds_lsm['sftlf'].values, (NT,1,1)) >= 100

# Mask the tasmax, retain only data over land
da_tasmax_masked = ds_tasmax['tasmax'].where(mask_condition)

# Compute spatial mean of tasmax over land
da_tasmax_spatial_av = da_tasmax_masked.mean(dim=['lat', 'lon'])

Plot timeseries of tasmax averaged over Sydney region

In [ ]:
fig = plt.figure(figsize=(10, 3))

# 
t0 = datetime(2017, 2, 7)
t1 = datetime(2017, 2, 14)
#
# First subfigure showing the temperature changes during the whole period
# 
ax = plt.subplot(1, 2, 1)
da_tasmax_spatial_av.plot.line()
plt.axvline(x=t0, color='r')
plt.axvline(x=t1, color='r')

#
# Second subfigure showing the temperature changes 
# 
ax = plt.subplot(1, 2, 2)
da_tasmax_spatial_av.sel(time=slice(t0, t1)).plot.line()

### 3. Examine the evolution of the first heatwave in February 2017

Define domain and time of interests as per above

In [ ]:
# Location of Sydney
loc = (-33.8688, 151.2093)

tstart = 20170208
tend = 20170214

# Look at a larger 10x10 deg domain centered at Sydney
latmin_r = loc[0] - 5
latmax_r = loc[0] + 5
lonmin_r = loc[1] - 5
lonmax_r = loc[1] + 5

Load all the data for different variables

In [ ]:
#
# Static variables
#

# Load the orography data
ds_orog = barra2.load_barra2_data("BARRA-R2", "fx", "orog", 
                                  latrange=(latmin_r, latmax_r),
                                  lonrange=(lonmin_r, lonmax_r))
# Load land sea mask
ds_lsm = barra2.load_barra2_data("BARRA-R2", "fx", "sftlf",
                                 latrange=(latmin_r, latmax_r),
                                 lonrange=(lonmin_r, lonmax_r))

In [ ]:
#
# Time varying variables
#
# If unclear what these variables are, use loaddata.whatis(freq, variable_name)
# 
ds_tasmax = barra2.load_barra2_data("BARRA-R2", "1hr", "tasmax",
                                    tstart=tstart, tend=tend,
                                    latrange=(latmin_r, latmax_r),
                                    lonrange=(lonmin_r, lonmax_r),
                                    chunks={'time': 'auto'})
ds_uas = barra2.load_barra2_data("BARRA-R2", "1hr", "uas",
                                 tstart=tstart, tend=tend,
                                 latrange=(latmin_r, latmax_r),
                                 lonrange=(lonmin_r, lonmax_r),
                                 chunks={'time': 'auto'})
ds_vas = barra2.load_barra2_data("BARRA-R2", "1hr", "vas",
                                 tstart=tstart, tend=tend,
                                 latrange=(latmin_r, latmax_r),
                                 lonrange=(lonmin_r, lonmax_r),
                                 chunks={'time': 'auto'})
ds_cll = barra2.load_barra2_data("BARRA-R2", "1hr", "cll",
                                 tstart=tstart, tend=tend,
                                 latrange=(latmin_r, latmax_r),
                                 lonrange=(lonmin_r, lonmax_r),
                                 chunks={'time': 'auto'})

Pre-process the data for plotting

Convert temperature to degC

In [ ]:
# Convert temperature from K to degC
ds_tasmax['tasmax' ] = ds_tasmax['tasmax'] - 273.15
ds_tasmax['tasmax'] = ds_tasmax['tasmax'].assign_attrs({"units": "degC"})

(NT, NY, NX) = ds_tasmax['tasmax'].shape

Combine 10 metre wind components, uas and vas, into a single xr.Dataset object so that xarray can plot both as vector field

In [ ]:
# Combine into a single xr.Dataset object
ds_uv = xr.merge([ds_uas['uas'], ds_vas['vas']])

# Thin the data horizontally for plotting purposes, to avoid having very densed wind vectors
ds_uv_subsampled = ds_uv.isel(lat=range(0, NY, 7), lon=range(0, NX, 7)).compute()

Compute spatial mean of tasmax over the smaller subdomain around Sydney

In [ ]:
# Define land sea mask over a small 2x2 degrees bounding box around Sydney
latmin_c = loc[0] - 1
latmax_c = loc[0] + 1
lonmin_c = loc[1] - 1
lonmax_c = loc[1] + 1

ds_lsm_c = ds_lsm.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))
mask_condition = (np.tile(ds_lsm_c['sftlf'], (NT,1,1)) >= 100)

# Truncate over a smaller domain focussing around Sydney
ds_tasmax_c = ds_tasmax.sel(lat=slice(latmin_c, latmax_c), lon=slice(lonmin_c, lonmax_c))

# Apply land sea mask to compute spatial mean over land points only
ds_tasmax_masked = ds_tasmax_c['tasmax'].where(mask_condition)

da_tasmax_spatial_av = ds_tasmax_masked.mean(dim=['lat', 'lon'])

Plot the heatwave with a series of 3-panels

Each 3-panel shows,
- Left: Timeseries of tasmax around Sydney
- Middle: Spatial maps of tasmax and 10 m wind
- Right: Spatial maps of cloud cover

In [ ]:
for time_step in range(0, NT, 3):
    # Plotting for every 3 time steps, i.e., 3-hourly snapshots
    
    # Plotting for this time step
    t = ds_tasmax['time'][time_step]
    
    # Set up the figure object
    fig = plt.figure(figsize=(14, 4))
    
    #
    # First subfigure showing the timeseries of spatial mean tasmax
    #
    ax1 = plt.subplot(1, 3, 1)
    da_tasmax_spatial_av.plot.line()
    ax1.plot(t.data, da_tasmax_spatial_av.data[time_step], 'or')
    # label the time step
    ax1.set_title(t.data)
    
    #
    # Second subfigure plotting temperature and wind vectors
    #
    ax2 = plt.subplot(1, 3, 2)
    # Plot tasmax as background
    ds_tasmax.sel(time=t, method='nearest')['tasmax'].plot(vmin=0, vmax=40, cmap='RdBu_r', cbar_kwargs={"shrink": 0.5})
    # Plot the 10m wind vector
    ds_uv_subsampled.sel(time=t, method='nearest').plot.quiver(x='lon', y='lat', u='uas', v='vas', color='blue')
    # Plot surface altitude as contour
    ds_orog['orog'].plot.contour(levels=4, colors='k')
    # Indicate where is Sydney
    ax2.plot(loc[1], loc[0], 'xr', markersize=15)
    
    #
    # Last subfigure plotting low level cloud
    #
    ax3 = plt.subplot(1, 3, 3)
    # Plot cloud 
    ds_cll['cll'].sel(time=t, method='nearest').plot(vmin=0, vmax=100, cmap='Greys_r', cbar_kwargs={"shrink": 0.5})
    # Plot surface altitude as contour
    ds_orog['orog'].plot.contour(levels=4, colors='b')
    # Indicate where is Sydney
    ax3.plot(loc[1], loc[0], 'xr', markersize=15)
    
    fig.tight_layout()
    
# this may take a while!

In [ ]:
client.close()

### 4. Exercise: Consider a different high impact weather event. 

Examples are,
1. Melbourne dust storm on 8 February 1983, "“everything went black" as the result of dry, cold front crossing Victoria, preceded by hot, gusty northerly winds. https://webarchive.nla.gov.au/awa/20090330051442/http://pandora.nla.gov.au/pan/96122/20090317-1643/www.bom.gov.au/lam/climate/levelthree/c20thc/storm7.html Consider look at temperature (1hr/tas), wind (1hr/uas, vas) and top-layer soil moisture (1hr/mrsos)

2. Sydney hailstorm on 14 April 1999, with most affected areas include south-east suburbs of Kensington, Kingsford, Botany, Mascot, Randwick and Paddington. https://knowledge.aidr.org.au/resources/storm-sydney-1999/ Consider look at precipitation (1hr/pr) and CAPE (1hr/CAPE).

3. Extreme rainfall in NSW over 2021/2022 - compound event where a series of mesoscale rainfall occurred sequentially in the same location. http://www.bom.gov.au/climate/current/statements/scs76.pdf?20220525 Consider look at precipitation (1hr/pr), sea-level pressure (1hr/psl), and soil moisture (1hr/mrsos or 3hr/mrsol)

4. Black summer bushfire 2019/2020 in SE Australia, during September 2019 to February 2020. http://www.bom.gov.au/climate/current/statements/scs73.pdf Consider look at temperature (1hr/tas), relative humidity (1hr/hurs) and wind (1hr/uas, vas).